In [1]:
# imports the required libraries

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import gc

In [2]:
#Only transformation carried out is converting to tensor. No other transformations are done
train_augmentations = transforms.Compose([transforms.ToTensor()])

test_augmentations = transforms.Compose([transforms.ToTensor()])

In [3]:
path = '/tmp'

train_ds = datasets.CIFAR10(root=path,
                            train=True,
                            transform=train_augmentations,
                            download=True
                           )

test_ds = datasets.CIFAR10(root=path,
                           train=False,
                           transform=test_augmentations
                           )

labels = 'airplane automobile bird cat deer dog frog horse ship truck'.split()

train_dataset = torch.utils.data.Subset(train_ds, [i for i in range(25000)])

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /tmp/cifar-10-python.tar.gz to /tmp


In [4]:
len(train_dataset), len(test_ds)

(25000, 10000)

In [5]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=256, shuffle=True, num_workers=2)

In [6]:
class Custom_Network(nn.Module):
    
    def __init__(self, in_features=3, num_classes=10):
        super(Custom_Network, self).__init__()
        
        self.conv_block = nn.Sequential( nn.Conv2d(in_channels=in_features,out_channels=6,kernel_size=3,stride=1),
                                         nn.ReLU(),
                                         nn.MaxPool2d(2,2),
                                         nn.Conv2d(in_channels=6,out_channels=12,kernel_size=3,stride=1),
                                         nn.ReLU(),
                                         nn.Conv2d(in_channels=12,out_channels=16,kernel_size=3,stride=1),
                                         nn.ReLU(),
                                         nn.MaxPool2d(2,2)
                                        )
        
        self.linear_block = nn.Sequential( nn.Linear(16*5*5, 120),
                                           nn.ReLU(),
                                           nn.Linear(120,84),
                                           nn.ReLU(),
                                           nn.Linear(84,10),
                                           nn.ReLU()
                                         )
        
    def forward(self, x):
        x = self.conv_block(x)
        x = torch.flatten(x,1)
        x = self.linear_block(x)
        return x


In [7]:
# CUDA
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [8]:
model = Custom_Network().to(device)
print(model)

Custom_Network(
  (conv_block): Sequential(
    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): Conv2d(12, 16, kernel_size=(3, 3), stride=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_block): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [41]:
class Trainer:
    def __init__(self, model, dataloaders, device):
        
        self.config = {
            'lr':1e-3,
            'epochs': 10
        }
        
        self.model = model
        self.train_loader, self.test_loader = dataloaders
        self.loss_fn = nn.CrossEntropyLoss()
        self.optim = torch.optim.Adam(self.model.parameters(), lr = self.config['lr'])
        self.device = device
        
    def accuracy(self, output, y):
        pred_labels = torch.argmax(output, dim=1)
        return (pred_labels == y).sum().item() / len(y)
        
    def train_one_epoch(self,regulariser):
        running_loss = 0
        running_acc = 0
        lambda_l1=0.001
        lambda_l2=0.001
        reg_loss=0
        for x,y in self.train_loader:
            self.optim.zero_grad()
            x = x.to(self.device, dtype=torch.float)
            y = y.to(self.device, dtype=torch.long)
            output = self.model(x)
            loss = self.loss_fn(output, y)
            if regulariser=='L1':
              reg_loss = sum(p.abs().sum()for p in model.parameters())
              loss +=lambda_l1*reg_loss
            elif regulariser=='L2':
              reg_loss = sum(p.pow(2.0).sum()for p in model.parameters())
              loss +=lambda_l2*reg_loss
            else:
              loss = self.loss_fn(output, y)
            loss.backward()
            self.optim.step()
            running_loss += loss.item()
            running_acc += self.accuracy(output,y)       
            del x,y,output   
        train_loss = running_loss/len(self.train_loader)
        train_acc = running_acc/len(self.train_loader) 
        return train_loss, train_acc
    
    @torch.no_grad()
    def valid_one_epoch(self):
        
        running_loss = 0
        running_acc = 0
        
        for x,y in self.test_loader:
            
            x = x.to(self.device, dtype=torch.float)
            y = y.to(self.device, dtype=torch.long)
            
            output = self.model(x)
            
            loss = self.loss_fn(output, y)
            
            running_loss += loss.item()
            running_acc += self.accuracy(output,y)
            
            del x,y,output
            
        test_loss = running_loss/len(self.test_loader)
        test_acc = running_acc/len(self.test_loader)
        
        return test_loss, test_acc
    
    
    def fit(self,regulariser):        
        train_losses,train_accs = [], []
        test_losses, test_accs = [], []       
        for epoch in range(self.config['epochs']):
            print(f"Model is using {'cuda' if next(self.model.parameters()).is_cuda else 'cpu'}")
            '''
            This is a very important step while using dropout. This tells the model that it is in training model. For network regularization,
            dropout is only used while training and not while evaluation.
            '''
            self.model.train() 
            train_loss, train_acc = self.train_one_epoch(regulariser)
            tb.add_scalars("Loss With Regularization/Training_Loss", {'train_loss_L2_Reg':train_loss}, epoch)
            tb.add_scalars("Accuracy With Regularization/Training_Accuracy", {'train_acc_L2_Reg':train_acc}, epoch)
            train_losses.append(train_loss)
            train_accs.append(train_acc)     
            self.model.eval()            
            test_loss, test_acc = self.valid_one_epoch()
            tb.add_scalars("Loss With Regularization/Testing_Loss", {'test_loss_L2_Reg':test_loss}, epoch)
            tb.add_scalars("Accuracy With Regularization/Testing_Accuracy", {'test_acc_L2_Reg':test_acc}, epoch)
            test_losses.append(test_loss)
            test_accs.append(test_acc)            
            print(f"------EPOCH {epoch+1}/{self.config['epochs']}------")
            print(f"Training: LOSS: {train_loss} | ACCURACY: {train_acc}")
            print(f"Testing: LOSS: {test_loss} | ACCURACY: {test_acc}\n\n")           
            # CLEANUP
            gc.collect()
            torch.cuda.empty_cache()
            tb.flush()       
        return (train_losses, train_accs), (test_losses, test_accs)

In [13]:
from torch.utils.tensorboard import SummaryWriter
import torchvision

tb = SummaryWriter()
tb.close()

In [16]:
trainer = Trainer(model, (train_dataloader, test_dataloader), device)
regulariser="None"
(train_losses, train_accs), (test_losses, test_accs) = trainer.fit(regulariser)

Model is using cuda
------EPOCH 1/10------
Training: LOSS: 1.9995747670835378 | ACCURACY: 0.27994032434402333
Testing: LOSS: 1.952812922000885 | ACCURACY: 0.2912109375


Model is using cuda
------EPOCH 2/10------
Training: LOSS: 1.9201509538961916 | ACCURACY: 0.3136844023323615
Testing: LOSS: 1.8711986392736435 | ACCURACY: 0.33046875


Model is using cuda
------EPOCH 3/10------
Training: LOSS: 1.8336867094039917 | ACCURACY: 0.34865843658892126
Testing: LOSS: 1.801555296778679 | ACCURACY: 0.3626953125


Model is using cuda
------EPOCH 4/10------
Training: LOSS: 1.7706426564528017 | ACCURACY: 0.3712949465500486
Testing: LOSS: 1.731589612364769 | ACCURACY: 0.3912109375


Model is using cuda
------EPOCH 5/10------
Training: LOSS: 1.7277364511879123 | ACCURACY: 0.38897177174441205
Testing: LOSS: 1.7114948153495788 | ACCURACY: 0.39873046875


Model is using cuda
------EPOCH 6/10------
Training: LOSS: 1.713646022640929 | ACCURACY: 0.3972303206997085
Testing: LOSS: 1.6839230090379715 | ACCURAC

In [17]:
!tensorboard dev upload --logdir 'runs' --name "Baseline CIFAR 10 Data " --description "Baseline CIFAR10 Model"


***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

runs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&state=Woy2Kz3QPZW4NbsmiBdS9QWjyHowQd&prompt=consent&access_type=offline
Enter th

**Exercise 1: Normalization Effect (CNN)**

1. Data Augmentation: the process of artificially ’increasing’ our dataset by adding translation, scaling and flipping to the images to fabricate examples for training.
2. Normalization: Normalizing the input data helps remove the dataset artifacts that can cause poor model performance.





In [18]:
#Training Network with only Data Augmentations
train_augmentations1 = transforms.Compose([transforms.RandomGrayscale(0.2),
                                          transforms.RandomHorizontalFlip(0.5),
                                          transforms.RandomVerticalFlip(0.2),
                                          transforms.RandomRotation(30),
                                          transforms.RandomAdjustSharpness(0.4),
                                          transforms.ToTensor(),
                                         ])

test_augmentations1 = transforms.Compose([transforms.ToTensor()]
                                       )

In [19]:
path = '/tmp'

train_ds1 = datasets.CIFAR10(root=path,
                            train=True,
                            transform=train_augmentations1,
                            download=True
                           )

test_ds1 = datasets.CIFAR10(root=path,
                           train=False,
                           transform=test_augmentations1
                           )

labels = 'airplane automobile bird cat deer dog frog horse ship truck'.split()

Files already downloaded and verified


In [20]:
train_dataloader_with_aug = torch.utils.data.DataLoader(train_ds1, batch_size=256, shuffle=True, num_workers=2)
test_dataloader_with_aug = torch.utils.data.DataLoader(test_ds1, batch_size=256, shuffle=True, num_workers=2)

In [22]:
trainer_aug = Trainer(model, (train_dataloader_with_aug, test_dataloader_with_aug), device)

(train_losses, train_accs), (test_losses, test_accs) = trainer_aug.fit(regulariser)

Model is using cuda
------EPOCH 1/10------
Training: LOSS: 1.8310919641232004 | ACCURACY: 0.35395806760204085
Testing: LOSS: 1.6377997934818267 | ACCURACY: 0.4353515625


Model is using cuda
------EPOCH 2/10------
Training: LOSS: 1.8014435062603074 | ACCURACY: 0.36695631377551025
Testing: LOSS: 1.6327746242284775 | ACCURACY: 0.43759765625


Model is using cuda
------EPOCH 3/10------
Training: LOSS: 1.78185010078002 | ACCURACY: 0.3734135841836735
Testing: LOSS: 1.6189169466495514 | ACCURACY: 0.43681640625


Model is using cuda
------EPOCH 4/10------
Training: LOSS: 1.7656347471840528 | ACCURACY: 0.3810267857142857
Testing: LOSS: 1.6025858342647552 | ACCURACY: 0.44697265625


Model is using cuda
------EPOCH 5/10------
Training: LOSS: 1.752226401956714 | ACCURACY: 0.38722496811224494
Testing: LOSS: 1.5914525032043456 | ACCURACY: 0.451953125


Model is using cuda
------EPOCH 6/10------
Training: LOSS: 1.7391123418905297 | ACCURACY: 0.3897201849489796
Testing: LOSS: 1.5939214408397675 | ACC

In [23]:
!tensorboard dev upload --logdir 'runs' --name "Augmentation CIFAR 10 Data " --description "Augmentation CIFAR10 Model"

Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/LPc4GeSbRhqVGEoQv8kuQQ/

[2022-06-26T08:17:31] Started scanning logdir.
[2022-06-26T08:17:32] Total uploaded: 80 scalars, 0 tensors, 0 binary objects
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 312, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 258, in _run_main
    sys.exit(main(argv))
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/program.py", line 276, in main
    return runner(self.flags) or 0
  File "/usr/local/lib/python3.7/dis

In [24]:
#Training Network with Only Normalization
train_augmentations2 = transforms.Compose([transforms.ToTensor(),
                                          transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                         ])

test_augmentations2 = transforms.Compose([transforms.ToTensor(), 
                                         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
                                       )

In [25]:
path = '/tmp'

train_ds2 = datasets.CIFAR10(root=path,
                            train=True,
                            transform=train_augmentations2,
                            download=True
                           )

test_ds2 = datasets.CIFAR10(root=path,
                           train=False,
                           transform=test_augmentations2
                           )

labels = 'airplane automobile bird cat deer dog frog horse ship truck'.split()

Files already downloaded and verified


In [26]:
train_dataloader_with_normalization = torch.utils.data.DataLoader(train_ds2, batch_size=256, shuffle=True, num_workers=2)
test_dataloader_with_normalization = torch.utils.data.DataLoader(test_ds2, batch_size=256, shuffle=True, num_workers=2)

In [28]:
# model_norm = Custom_Network().to(device)
trainer_with_normalization= Trainer(model, (train_dataloader_with_normalization, test_dataloader_with_normalization), device)

(train_losses, train_accs), (test_losses, test_accs) = trainer_with_normalization.fit(regulariser)

Model is using cuda
------EPOCH 1/10------
Training: LOSS: 1.6999912109910225 | ACCURACY: 0.42553411989795914
Testing: LOSS: 1.5642434805631638 | ACCURACY: 0.456640625


Model is using cuda
------EPOCH 2/10------
Training: LOSS: 1.5235653726422056 | ACCURACY: 0.47338568239795914
Testing: LOSS: 1.5277952343225478 | ACCURACY: 0.471484375


Model is using cuda
------EPOCH 3/10------
Training: LOSS: 1.4815078018879404 | ACCURACY: 0.48635204081632655
Testing: LOSS: 1.5237304091453552 | ACCURACY: 0.46884765625


Model is using cuda
------EPOCH 4/10------
Training: LOSS: 1.4513276347092219 | ACCURACY: 0.4980030293367347
Testing: LOSS: 1.478224778175354 | ACCURACY: 0.48701171875


Model is using cuda
------EPOCH 5/10------
Training: LOSS: 1.4243237534347846 | ACCURACY: 0.5080875318877551
Testing: LOSS: 1.4724803656339644 | ACCURACY: 0.48623046875


Model is using cuda
------EPOCH 6/10------
Training: LOSS: 1.4002425530735327 | ACCURACY: 0.5145727040816327
Testing: LOSS: 1.4577158719301224 | AC

In [29]:
!tensorboard dev upload --logdir 'runs' --name "Normalisation CIFAR 10 Data " --description "Normalisation CIFAR10 Model"

Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/o75NKX3ZRieLsvdHLhoLxA/

[2022-06-26T08:26:58] Started scanning logdir.
[2022-06-26T08:26:58] Total uploaded: 120 scalars, 0 tensors, 0 binary objects


Interrupted. View your TensorBoard at https://tensorboard.dev/experiment/o75NKX3ZRieLsvdHLhoLxA/
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 312, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 258, in _run_main
    sys.exit(main(argv))
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/pr

**Exercise 2: Network Regularization (CNN)**

In [31]:
train_augmentations_all = transforms.Compose([transforms.RandomGrayscale(0.2),
                                          transforms.RandomHorizontalFlip(0.5),
                                          transforms.RandomVerticalFlip(0.2),
                                          transforms.RandomRotation(30),
                                          transforms.RandomAdjustSharpness(0.4),
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                         ])

test_augmentations_all = transforms.Compose([transforms.ToTensor(), 
                                         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
                                       )

In [32]:
path = '/tmp'

train_ds_all = datasets.CIFAR10(root=path,
                            train=True,
                            transform=train_augmentations_all,
                            download=True
                           )

test_ds_all = datasets.CIFAR10(root=path,
                           train=False,
                           transform=test_augmentations_all
                           )

labels = 'airplane automobile bird cat deer dog frog horse ship truck'.split()

Files already downloaded and verified


In [33]:
train_dataloader_all = torch.utils.data.DataLoader(train_ds_all, batch_size=256, shuffle=True, num_workers=2)
test_dataloader_all = torch.utils.data.DataLoader(test_ds_all, batch_size=256, shuffle=True, num_workers=2)

In [34]:
class Net_dropout(nn.Module):
    
    def __init__(self, in_features=3, num_classes=10):
        super(Net_dropout, self).__init__()
        
        self.conv_block = nn.Sequential( nn.Conv2d(in_channels=in_features,out_channels=6,kernel_size=3,stride=1),
                                         nn.ReLU(),
                                         nn.MaxPool2d(2,2),
                                         nn.Conv2d(in_channels=6,out_channels=12,kernel_size=3,stride=1),
                                         nn.ReLU(),
                                         nn.Conv2d(in_channels=12,out_channels=16,kernel_size=3,stride=1),
                                         nn.ReLU(),
                                         nn.MaxPool2d(2,2)
                                        )
        
        self.linear_block = nn.Sequential( nn.Linear(16*5*5, 120),
                                           nn.ReLU(),
                                           nn.Dropout(0.25),
                                           nn.Linear(120,84),
                                           nn.ReLU(),
                                           nn.Dropout(0.25),
                                           nn.Linear(84,10),
                                           nn.ReLU()

                                         )
        # self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        x = self.conv_block(x)
        x = torch.flatten(x,1)
        x = self.linear_block(x)
        return x


In [35]:
model = Net_dropout().to(device)
print(model)

Net_dropout(
  (conv_block): Sequential(
    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): Conv2d(12, 16, kernel_size=(3, 3), stride=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_block): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Linear(in_features=120, out_features=84, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.25, inplace=False)
    (6): Linear(in_features=84, out_features=10, bias=True)
    (7): ReLU()
  )
)


In [36]:
trainer = Trainer(model, (train_dataloader_all, test_dataloader_all), device)
regulariser="None"
(train_losses, train_accs), (test_losses, test_accs) = trainer.fit(regulariser)

Model is using cuda
------EPOCH 1/10------
Training: LOSS: 2.163170884458386 | ACCURACY: 0.22529496173469388
Testing: LOSS: 1.9181569248437882 | ACCURACY: 0.3501953125


Model is using cuda
------EPOCH 2/10------
Training: LOSS: 1.9826867598660138 | ACCURACY: 0.31368383290816326
Testing: LOSS: 1.7227968364953994 | ACCURACY: 0.40185546875


Model is using cuda
------EPOCH 3/10------
Training: LOSS: 1.8482156140463692 | ACCURACY: 0.34956951530612246
Testing: LOSS: 1.6514089226722717 | ACCURACY: 0.4244140625


Model is using cuda
------EPOCH 4/10------
Training: LOSS: 1.7993173690474764 | ACCURACY: 0.3680843431122449
Testing: LOSS: 1.5964259207248688 | ACCURACY: 0.44375


Model is using cuda
------EPOCH 5/10------
Training: LOSS: 1.7669338054802952 | ACCURACY: 0.3820232780612245
Testing: LOSS: 1.5742644906044005 | ACCURACY: 0.45107421875


Model is using cuda
------EPOCH 6/10------
Training: LOSS: 1.741213980377937 | ACCURACY: 0.3917291135204082
Testing: LOSS: 1.5423934876918792 | ACCURAC

In [43]:
!tensorboard dev upload --logdir 'runs' --name "Dropout CIFAR 10 Data " --description "Dropout CIFAR10 Model"

Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/z87QiQqmTyq7YEFycyDINg/

[2022-06-26T09:04:19] Started scanning logdir.
[2022-06-26T09:04:19] Total uploaded: 240 scalars, 0 tensors, 0 binary objects


Interrupted. View your TensorBoard at https://tensorboard.dev/experiment/z87QiQqmTyq7YEFycyDINg/
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 312, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 258, in _run_main
    sys.exit(main(argv))
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/pr

In [39]:
# For L1 Regularisation
trainer = Trainer(model, (train_dataloader_all, test_dataloader_all), device)
regulariser='L1'
(train_losses, train_accs), (test_losses, test_accs) = trainer.fit(regulariser)

Model is using cuda
------EPOCH 1/10------
Training: LOSS: 3.162023984656042 | ACCURACY: 0.41946348852040816
Testing: LOSS: 1.4132280141115188 | ACCURACY: 0.489453125


Model is using cuda
------EPOCH 2/10------
Training: LOSS: 2.365024064268385 | ACCURACY: 0.40707110969387755
Testing: LOSS: 1.444857546687126 | ACCURACY: 0.48564453125


Model is using cuda
------EPOCH 3/10------
Training: LOSS: 2.243377219657509 | ACCURACY: 0.39493781887755103
Testing: LOSS: 1.4606525719165802 | ACCURACY: 0.46953125


Model is using cuda
------EPOCH 4/10------
Training: LOSS: 2.1877212767698326 | ACCURACY: 0.3940808354591837
Testing: LOSS: 1.4607624381780624 | ACCURACY: 0.47216796875


Model is using cuda
------EPOCH 5/10------
Training: LOSS: 2.152590570401172 | ACCURACY: 0.39530851403061223
Testing: LOSS: 1.4677068769931794 | ACCURACY: 0.46318359375


Model is using cuda
------EPOCH 6/10------
Training: LOSS: 2.1228376675625236 | ACCURACY: 0.3992227359693878
Testing: LOSS: 1.4296285033226013 | ACCURA

In [42]:
# For L2 Regularisation
trainer = Trainer(model, (train_dataloader_all, test_dataloader_all), device)
regulariser='L2'
(train_losses, train_accs), (test_losses, test_accs) = trainer.fit(regulariser)

Model is using cuda
------EPOCH 1/10------
Training: LOSS: 1.6794119799623684 | ACCURACY: 0.4116828762755102
Testing: LOSS: 1.3893619000911712 | ACCURACY: 0.48701171875


Model is using cuda
------EPOCH 2/10------
Training: LOSS: 1.6447452768987538 | ACCURACY: 0.4275749362244898
Testing: LOSS: 1.3564866095781327 | ACCURACY: 0.50654296875


Model is using cuda
------EPOCH 3/10------
Training: LOSS: 1.624359213576025 | ACCURACY: 0.4353037308673469
Testing: LOSS: 1.3439359962940216 | ACCURACY: 0.51005859375


Model is using cuda
------EPOCH 4/10------
Training: LOSS: 1.6210942615051658 | ACCURACY: 0.4424545599489796
Testing: LOSS: 1.3233083873987197 | ACCURACY: 0.5234375


Model is using cuda
------EPOCH 5/10------
Training: LOSS: 1.5972214037058305 | ACCURACY: 0.4529296875
Testing: LOSS: 1.2785885602235794 | ACCURACY: 0.540234375


Model is using cuda
------EPOCH 6/10------
Training: LOSS: 1.5954015181989085 | ACCURACY: 0.45361527423469383
Testing: LOSS: 1.286544892191887 | ACCURACY: 0.5

### **Exercise 3: Optimizers**

 We test with two different Optimisers SGD and AdamOptimizer

In [45]:
learning_rate_array=[0.001,0.00001]
optimiser_arr=["Adam","SGD"]

In [56]:
class Trainer_op:
    def __init__(self, model, dataloaders, device,learning_rate,optimiser):
        
        self.config = {
            'lr':learning_rate,
            'epochs': 10
        }
        
        self.model = model
        self.train_loader, self.test_loader = dataloaders
        self.loss_fn = nn.CrossEntropyLoss()
        if optimiser=='Adam':
          self.optim = torch.optim.Adam(self.model.parameters(), lr = self.config['lr'])
        else:
          self.optim = torch.optim.SGD(self.model.parameters(), lr = self.config['lr'])
        self.device = device
        
    def accuracy(self, output, y):
        pred_labels = torch.argmax(output, dim=1)
        return (pred_labels == y).sum().item() / len(y)
        
    def train_one_epoch(self,regulariser):
        running_loss = 0
        running_acc = 0
        lambda_l1=0.001
        lambda_l2=0.001
        reg_loss=0
        for x,y in self.train_loader:
            self.optim.zero_grad()
            x = x.to(self.device, dtype=torch.float)
            y = y.to(self.device, dtype=torch.long)
            output = self.model(x)
            loss = self.loss_fn(output, y)
            if regulariser=='L1':
              # print("Inside L1 Regulariser")
              # for param in model.parameters():
              #   reg_loss+=param.abs().sum()
              reg_loss = sum(p.abs().sum()for p in model.parameters())
              loss +=lambda_l1*reg_loss
            elif regulariser=='L2':
              # print("Inside L2 Regulariser")
              reg_loss = sum(p.pow(2.0).sum()for p in model.parameters())
              # for param in model.parameters():
              #   reg_loss+=param.pow(2).sum()
              loss +=lambda_l2*reg_loss
            else:
              loss = self.loss_fn(output, y)
            

            loss.backward()
            self.optim.step()
            running_loss += loss.item()
            running_acc += self.accuracy(output,y)       
            del x,y,output   
        train_loss = running_loss/len(self.train_loader)
        train_acc = running_acc/len(self.train_loader) 
        return train_loss, train_acc
    
    @torch.no_grad()
    def valid_one_epoch(self):
        
        running_loss = 0
        running_acc = 0
        
        for x,y in self.test_loader:
            
            x = x.to(self.device, dtype=torch.float)
            y = y.to(self.device, dtype=torch.long)
            
            output = self.model(x)
            
            loss = self.loss_fn(output, y)
            
            running_loss += loss.item()
            running_acc += self.accuracy(output,y)
            
            del x,y,output
            
        test_loss = running_loss/len(self.test_loader)
        test_acc = running_acc/len(self.test_loader)
        
        return test_loss, test_acc
    
    
    def fit(self,regulariser,lr,op):
        
        train_losses,train_accs = [], []
        test_losses, test_accs = [], []
        
        for epoch in range(self.config['epochs']):
            
            print(f"Model is using {'cuda' if next(self.model.parameters()).is_cuda else 'cpu'}")
            self.model.train() 
            
            train_loss, train_acc = self.train_one_epoch(regulariser)

            tb.add_scalars("Loss with diff Optimiser and Learning Rate/Training_Loss", {'train_loss_SGD_0.00001':train_loss}, epoch)
            tb.add_scalars("Accuracy with diff Optimiser and Learning Rate/Training_Accuracy", {'train_acc_SGD_0.00001':train_acc}, epoch)

            train_losses.append(train_loss)
            train_accs.append(train_acc)
            
            self.model.eval()
            
            test_loss, test_acc = self.valid_one_epoch()

            tb.add_scalars("Loss with diff Optimiser and Learning Rate/Testing_Loss", {'test_loss_SGD_0.00001':test_loss}, epoch)
            tb.add_scalars("Accuracy with diff Optimiser and Learning Rate/Testing_Accuracy", {'test_acc_SGD_0.00001':test_acc}, epoch)

            test_losses.append(test_loss)
            test_accs.append(test_acc)
            
            print(f"------EPOCH for {op} Optimiser with Learning Rate {lr} {epoch+1}/{self.config['epochs']}------")
            print(f"Training: LOSS: {train_loss} | ACCURACY: {train_acc}")
            print(f"Testing: LOSS: {test_loss} | ACCURACY: {test_acc}\n\n")
            
            # CLEANUP
            gc.collect()
            torch.cuda.empty_cache()
            tb.flush()
        
        return (train_losses, train_accs), (test_losses, test_accs)

In [47]:
trainer = Trainer_op(model, (train_dataloader_all, test_dataloader_all), device,learning_rate_array[0],optimiser_arr[0])
(train_losses, train_accs), (test_losses, test_accs) = trainer.fit(regulariser,learning_rate_array[0],optimiser_arr[0])

Model is using cuda
------EPOCH for Adam Optimiser with Learning Rate 0.001 1/10------
Training: LOSS: 1.568271853485886 | ACCURACY: 0.4672632334183674
Testing: LOSS: 1.2520844906568527 | ACCURACY: 0.55107421875


Model is using cuda
------EPOCH for Adam Optimiser with Learning Rate 0.001 2/10------
Training: LOSS: 1.5593552267064854 | ACCURACY: 0.47099808673469384
Testing: LOSS: 1.2394691467285157 | ACCURACY: 0.5546875


Model is using cuda
------EPOCH for Adam Optimiser with Learning Rate 0.001 3/10------
Training: LOSS: 1.5599221866957995 | ACCURACY: 0.47494818239795916
Testing: LOSS: 1.242810446023941 | ACCURACY: 0.54853515625


Model is using cuda
------EPOCH for Adam Optimiser with Learning Rate 0.001 4/10------
Training: LOSS: 1.5608928240075404 | ACCURACY: 0.47167171556122445
Testing: LOSS: 1.227433317899704 | ACCURACY: 0.56162109375


Model is using cuda
------EPOCH for Adam Optimiser with Learning Rate 0.001 5/10------
Training: LOSS: 1.5497294627890295 | ACCURACY: 0.47648676

In [53]:
trainer = Trainer_op(model, (train_dataloader_all, test_dataloader_all), device,learning_rate_array[0],optimiser_arr[1])
(train_losses, train_accs), (test_losses, test_accs) = trainer.fit(regulariser,learning_rate_array[0],optimiser_arr[1])

Model is using cuda
------EPOCH for SGD Optimiser with Learning Rate 0.001 1/10------
Training: LOSS: 1.5004704764911108 | ACCURACY: 0.4995854591836735
Testing: LOSS: 1.1690899759531022 | ACCURACY: 0.57939453125


Model is using cuda
------EPOCH for SGD Optimiser with Learning Rate 0.001 2/10------
Training: LOSS: 1.49948372707075 | ACCURACY: 0.4980508609693878
Testing: LOSS: 1.1637003898620606 | ACCURACY: 0.58486328125


Model is using cuda
------EPOCH for SGD Optimiser with Learning Rate 0.001 3/10------
Training: LOSS: 1.5016942261433115 | ACCURACY: 0.4974968112244898
Testing: LOSS: 1.1748516321182252 | ACCURACY: 0.57685546875


Model is using cuda
------EPOCH for SGD Optimiser with Learning Rate 0.001 4/10------
Training: LOSS: 1.4945509020163088 | ACCURACY: 0.49851721938775506
Testing: LOSS: 1.1736849755048753 | ACCURACY: 0.57734375


Model is using cuda
------EPOCH for SGD Optimiser with Learning Rate 0.001 5/10------
Training: LOSS: 1.4978796420048694 | ACCURACY: 0.5001753826530

In [50]:
trainer = Trainer_op(model, (train_dataloader_all, test_dataloader_all), device,learning_rate_array[1],optimiser_arr[0])
(train_losses, train_accs), (test_losses, test_accs) = trainer.fit(regulariser,learning_rate_array[1],optimiser_arr[0])

Model is using cuda
------EPOCH for Adam Optimiser with Learning Rate 1e-05 1/10------
Training: LOSS: 1.5247197607342078 | ACCURACY: 0.48878348214285716
Testing: LOSS: 1.1826829224824906 | ACCURACY: 0.5767578125


Model is using cuda
------EPOCH for Adam Optimiser with Learning Rate 1e-05 2/10------
Training: LOSS: 1.5099038116785946 | ACCURACY: 0.49233896683673467
Testing: LOSS: 1.1768426418304443 | ACCURACY: 0.573046875


Model is using cuda
------EPOCH for Adam Optimiser with Learning Rate 1e-05 3/10------
Training: LOSS: 1.5068278209287294 | ACCURACY: 0.4962691326530612
Testing: LOSS: 1.1732120990753174 | ACCURACY: 0.5775390625


Model is using cuda
------EPOCH for Adam Optimiser with Learning Rate 1e-05 4/10------
Training: LOSS: 1.5029672256537847 | ACCURACY: 0.49624920280612245
Testing: LOSS: 1.1658751621842385 | ACCURACY: 0.58173828125


Model is using cuda
------EPOCH for Adam Optimiser with Learning Rate 1e-05 5/10------
Training: LOSS: 1.5035096358279794 | ACCURACY: 0.49509

In [57]:
trainer = Trainer_op(model, (train_dataloader_all, test_dataloader_all), device,learning_rate_array[1],optimiser_arr[1])
(train_losses, train_accs), (test_losses, test_accs) = trainer.fit(regulariser,learning_rate_array[1],optimiser_arr[1])

Model is using cuda
------EPOCH for SGD Optimiser with Learning Rate 1e-05 1/10------
Training: LOSS: 1.4917641634843788 | ACCURACY: 0.5016342474489796
Testing: LOSS: 1.1782564312219619 | ACCURACY: 0.57685546875


Model is using cuda
------EPOCH for SGD Optimiser with Learning Rate 1e-05 2/10------
Training: LOSS: 1.497227593344085 | ACCURACY: 0.4993064413265306
Testing: LOSS: 1.1678504675626755 | ACCURACY: 0.58115234375


Model is using cuda
------EPOCH for SGD Optimiser with Learning Rate 1e-05 3/10------
Training: LOSS: 1.4960670471191406 | ACCURACY: 0.4994419642857143
Testing: LOSS: 1.165156352519989 | ACCURACY: 0.5828125


Model is using cuda
------EPOCH for SGD Optimiser with Learning Rate 1e-05 4/10------
Training: LOSS: 1.4958208799362183 | ACCURACY: 0.4989078443877551
Testing: LOSS: 1.1712672680616378 | ACCURACY: 0.5794921875


Model is using cuda
------EPOCH for SGD Optimiser with Learning Rate 1e-05 5/10------
Training: LOSS: 1.4966949340032072 | ACCURACY: 0.5010841836734694

In [58]:
!tensorboard dev upload --logdir 'runs' --name "Optimiser Performance CIFAR 10 Data " --description "Optimiser Performance CIFAR10 Model"

Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/XL9ywNvvQtasB7fn5hyzOg/

[2022-06-26T09:40:37] Started scanning logdir.
[2022-06-26T09:40:37] Total uploaded: 400 scalars, 0 tensors, 0 binary objects


Interrupted. View your TensorBoard at https://tensorboard.dev/experiment/XL9ywNvvQtasB7fn5hyzOg/
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 312, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 258, in _run_main
    sys.exit(main(argv))
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/pr